In [2]:
import json
import logging
import os
import time

import mxnet as mx
from mxnet import autograd as ag
from mxnet import gluon
from mxnet.gluon.model_zoo import vision as models

/home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


## Downloading CIFAR10 dataset

In [ ]:
from cifar10_utils import download_training_data
download_training_data()

## Preparing Data

In [3]:
hyperparameters={'batch_size': 20, 
                           'epochs': 1, 
                           'learning_rate': 0.1, 
                           'momentum': 0.9}

def get_data(path, augment, num_cpus, batch_size, data_shape, resize=-1, num_parts=1, part_index=0):
    return mx.io.ImageRecordIter(
        path_imgrec=path,
        resize=resize,
        data_shape=data_shape,
        batch_size=batch_size,
        rand_crop=augment,
        rand_mirror=augment,
        preprocess_threads=num_cpus,
        num_parts=num_parts,
        part_index=part_index)


def get_test_data(num_cpus, data_dir, batch_size, data_shape, resize=-1):
    return get_data(os.path.join(data_dir, "test.rec"), False, num_cpus, batch_size, data_shape, resize, 1, 0)


def get_train_data(num_cpus, data_dir, batch_size, data_shape, resize=-1, num_parts=1, part_index=0):
    return get_data(os.path.join(data_dir, "train.rec"), True, num_cpus, batch_size, data_shape, resize, num_parts,
                    part_index)

def test(ctx, net, test_data):
    test_data.reset()
    metric = mx.metric.Accuracy()

    for i, batch in enumerate(test_data):
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    return metric.get()


## Loading Resnet model

In [22]:
hosts = [1]
part_index = 0
data_dir = "data"
model_dir = "model"
num_cpus = 2
batch_size = hyperparameters.get('batch_size', 128)
train_data = get_train_data(num_cpus, data_dir, batch_size, (3, 32, 32),
                            num_parts=len(hosts), part_index=part_index)
test_data = get_test_data(num_cpus, data_dir, batch_size, (3, 32, 32))

num_gpus = 1

batch_size = hyperparameters.get('batch_size', 128)
epochs = hyperparameters.get('epochs', 100)
learning_rate = hyperparameters.get('learning_rate', 0.1)
momentum = hyperparameters.get('momentum', 0.9)
log_interval = hyperparameters.get('log_interval', 1)
wd = hyperparameters.get('wd', 0.0001)

if len(hosts) == 1:
    kvstore = 'device' if num_gpus > 0 else 'local'
else:
    kvstore = 'dist_device_sync'

ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
net = models.get_model('resnet34_v2', ctx=ctx, pretrained=False, classes=10)
batch_size *= max(1, len(ctx))

## Transfer Learn the Network

In [4]:
    # Collect all parameters from net and its children, then initialize them.
    net.initialize(mx.init.Xavier(magnitude=2), ctx=ctx)
    # Trainer is for updating parameters with gradient.
    trainer = gluon.Trainer(net.collect_params(), 'sgd',
                            optimizer_params={'learning_rate': learning_rate, 'momentum': momentum, 'wd': wd},
                            kvstore=kvstore)
    metric = mx.metric.Accuracy()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()

    best_accuracy = 0.0
    for epoch in range(epochs):
        # reset data iterator and metric at begining of epoch.
        train_data.reset()
        tic = time.time()
        metric.reset()
        btic = time.time()

        for i, batch in enumerate(train_data):
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            Ls = []
            with ag.record():
                for x, y in zip(data, label): 
                    z = net(x)
                    L = loss(z, y)
                    # store the loss and do backward after we have done forward
                    # on all GPUs for better speed on multiple GPUs.
                    Ls.append(L)
                    outputs.append(z)
                for L in Ls:
                    L.backward()
            trainer.step(batch.data[0].shape[0])
            metric.update(label, outputs)
            if i % log_interval == 0 and i > 0:
                name, acc = metric.get()
                print('Epoch [%d] Batch [%d]\tSpeed: %f samples/sec\t%s=%f' %
                             (epoch, i, batch_size / (time.time() - btic), name, acc))
            btic = time.time()
        name, acc = metric.get()
        print('[Epoch %d] training: %s=%f' % (epoch, name, acc))
        print('[Epoch %d] time cost: %f' % (epoch, time.time() - tic))

        name, val_acc = test(ctx, net, test_data)
        print('[Epoch %d] validation: %s=%f' % (epoch, name, val_acc))
        

Epoch [0] Batch [1]	Speed: 239.059789 samples/sec	accuracy=0.100000
Epoch [0] Batch [2]	Speed: 228.681469 samples/sec	accuracy=0.100000
Epoch [0] Batch [3]	Speed: 225.242881 samples/sec	accuracy=0.112500
Epoch [0] Batch [4]	Speed: 234.719912 samples/sec	accuracy=0.100000
Epoch [0] Batch [5]	Speed: 232.175254 samples/sec	accuracy=0.108333
Epoch [0] Batch [6]	Speed: 234.416879 samples/sec	accuracy=0.107143
Epoch [0] Batch [7]	Speed: 231.892212 samples/sec	accuracy=0.093750
Epoch [0] Batch [8]	Speed: 240.119078 samples/sec	accuracy=0.116667
Epoch [0] Batch [9]	Speed: 236.919479 samples/sec	accuracy=0.120000
Epoch [0] Batch [10]	Speed: 242.068922 samples/sec	accuracy=0.118182
Epoch [0] Batch [11]	Speed: 243.712482 samples/sec	accuracy=0.120833
Epoch [0] Batch [12]	Speed: 237.247808 samples/sec	accuracy=0.123077
Epoch [0] Batch [13]	Speed: 239.839661 samples/sec	accuracy=0.117857
Epoch [0] Batch [14]	Speed: 241.589978 samples/sec	accuracy=0.110000
Epoch [0] Batch [15]	Speed: 243.581563 samp

Epoch [0] Batch [121]	Speed: 236.975700 samples/sec	accuracy=0.105738
Epoch [0] Batch [122]	Speed: 241.438394 samples/sec	accuracy=0.106504
Epoch [0] Batch [123]	Speed: 240.453584 samples/sec	accuracy=0.106452
Epoch [0] Batch [124]	Speed: 241.449513 samples/sec	accuracy=0.106400
Epoch [0] Batch [125]	Speed: 240.795244 samples/sec	accuracy=0.106349
Epoch [0] Batch [126]	Speed: 244.369908 samples/sec	accuracy=0.105906
Epoch [0] Batch [127]	Speed: 241.902785 samples/sec	accuracy=0.105859
Epoch [0] Batch [128]	Speed: 241.761956 samples/sec	accuracy=0.105426
Epoch [0] Batch [129]	Speed: 242.921331 samples/sec	accuracy=0.104615
Epoch [0] Batch [130]	Speed: 240.677106 samples/sec	accuracy=0.104198
Epoch [0] Batch [131]	Speed: 242.818669 samples/sec	accuracy=0.104167
Epoch [0] Batch [132]	Speed: 238.123987 samples/sec	accuracy=0.103383
Epoch [0] Batch [133]	Speed: 241.341842 samples/sec	accuracy=0.103358
Epoch [0] Batch [134]	Speed: 242.005372 samples/sec	accuracy=0.104074
Epoch [0] Batch [135

Epoch [0] Batch [241]	Speed: 247.997304 samples/sec	accuracy=0.108471
Epoch [0] Batch [242]	Speed: 245.603380 samples/sec	accuracy=0.108642
Epoch [0] Batch [243]	Speed: 240.392946 samples/sec	accuracy=0.108402
Epoch [0] Batch [244]	Speed: 243.021968 samples/sec	accuracy=0.107959
Epoch [0] Batch [245]	Speed: 240.612905 samples/sec	accuracy=0.107724
Epoch [0] Batch [246]	Speed: 244.920716 samples/sec	accuracy=0.108300
Epoch [0] Batch [247]	Speed: 246.411402 samples/sec	accuracy=0.108468
Epoch [0] Batch [248]	Speed: 247.659773 samples/sec	accuracy=0.108233
Epoch [0] Batch [249]	Speed: 244.266018 samples/sec	accuracy=0.108200
Epoch [0] Batch [250]	Speed: 242.683091 samples/sec	accuracy=0.107769
Epoch [0] Batch [251]	Speed: 248.725272 samples/sec	accuracy=0.107738
Epoch [0] Batch [252]	Speed: 243.067036 samples/sec	accuracy=0.107905
Epoch [0] Batch [253]	Speed: 247.310035 samples/sec	accuracy=0.107677
Epoch [0] Batch [254]	Speed: 250.303099 samples/sec	accuracy=0.107647
Epoch [0] Batch [255

Epoch [0] Batch [361]	Speed: 245.468267 samples/sec	accuracy=0.110359
Epoch [0] Batch [362]	Speed: 246.071675 samples/sec	accuracy=0.110744
Epoch [0] Batch [363]	Speed: 245.594751 samples/sec	accuracy=0.110714
Epoch [0] Batch [364]	Speed: 245.414407 samples/sec	accuracy=0.110685
Epoch [0] Batch [365]	Speed: 245.763840 samples/sec	accuracy=0.110792
Epoch [0] Batch [366]	Speed: 250.309821 samples/sec	accuracy=0.110627
Epoch [0] Batch [367]	Speed: 246.144601 samples/sec	accuracy=0.110462
Epoch [0] Batch [368]	Speed: 245.407945 samples/sec	accuracy=0.110840
Epoch [0] Batch [369]	Speed: 246.703448 samples/sec	accuracy=0.110811
Epoch [0] Batch [370]	Speed: 250.890765 samples/sec	accuracy=0.110512
Epoch [0] Batch [371]	Speed: 245.368465 samples/sec	accuracy=0.110618
Epoch [0] Batch [372]	Speed: 248.839633 samples/sec	accuracy=0.110724
Epoch [0] Batch [373]	Speed: 248.135216 samples/sec	accuracy=0.111096
Epoch [0] Batch [374]	Speed: 250.539331 samples/sec	accuracy=0.111067
Epoch [0] Batch [375

Epoch [0] Batch [481]	Speed: 245.850994 samples/sec	accuracy=0.119398
Epoch [0] Batch [482]	Speed: 248.564605 samples/sec	accuracy=0.119669
Epoch [0] Batch [483]	Speed: 249.202598 samples/sec	accuracy=0.119731
Epoch [0] Batch [484]	Speed: 245.694018 samples/sec	accuracy=0.119588
Epoch [0] Batch [485]	Speed: 246.029094 samples/sec	accuracy=0.119650
Epoch [0] Batch [486]	Speed: 243.941398 samples/sec	accuracy=0.119507
Epoch [0] Batch [487]	Speed: 246.635991 samples/sec	accuracy=0.119672
Epoch [0] Batch [488]	Speed: 250.178435 samples/sec	accuracy=0.119939
Epoch [0] Batch [489]	Speed: 249.242581 samples/sec	accuracy=0.119898
Epoch [0] Batch [490]	Speed: 245.407945 samples/sec	accuracy=0.119959
Epoch [0] Batch [491]	Speed: 250.387523 samples/sec	accuracy=0.120224
Epoch [0] Batch [492]	Speed: 245.570307 samples/sec	accuracy=0.120183
Epoch [0] Batch [493]	Speed: 249.662438 samples/sec	accuracy=0.120243
Epoch [0] Batch [494]	Speed: 249.631234 samples/sec	accuracy=0.120202
Epoch [0] Batch [495

...

Epoch [0] Batch [2131]	Speed: 252.069870 samples/sec	accuracy=0.174836
Epoch [0] Batch [2132]	Speed: 248.716422 samples/sec	accuracy=0.174871
Epoch [0] Batch [2133]	Speed: 252.003232 samples/sec	accuracy=0.174859
Epoch [0] Batch [2134]	Speed: 250.595465 samples/sec	accuracy=0.174871
Epoch [0] Batch [2135]	Speed: 256.265901 samples/sec	accuracy=0.174860
Epoch [0] Batch [2136]	Speed: 248.480669 samples/sec	accuracy=0.174871
Epoch [0] Batch [2137]	Speed: 249.289985 samples/sec	accuracy=0.174930
Epoch [0] Batch [2138]	Speed: 251.822428 samples/sec	accuracy=0.174918
Epoch [0] Batch [2139]	Speed: 252.633151 samples/sec	accuracy=0.174953
Epoch [0] Batch [2140]	Speed: 250.903522 samples/sec	accuracy=0.174918
Epoch [0] Batch [2141]	Speed: 251.730990 samples/sec	accuracy=0.175000
Epoch [0] Batch [2142]	Speed: 249.339631 samples/sec	accuracy=0.174988
Epoch [0] Batch [2143]	Speed: 252.407385 samples/sec	accuracy=0.175023
Epoch [0] Batch [2144]	Speed: 248.664076 samples/sec	accuracy=0.175058
Epoch 

Epoch [0] Batch [2248]	Speed: 246.545382 samples/sec	accuracy=0.177079
Epoch [0] Batch [2249]	Speed: 247.705846 samples/sec	accuracy=0.177111
Epoch [0] Batch [2250]	Speed: 253.218828 samples/sec	accuracy=0.177143
Epoch [0] Batch [2251]	Speed: 251.904855 samples/sec	accuracy=0.177176
Epoch [0] Batch [2252]	Speed: 254.780060 samples/sec	accuracy=0.177164
Epoch [0] Batch [2253]	Speed: 251.512866 samples/sec	accuracy=0.177152
Epoch [0] Batch [2254]	Speed: 247.812683 samples/sec	accuracy=0.177184
Epoch [0] Batch [2255]	Speed: 248.973312 samples/sec	accuracy=0.177150
Epoch [0] Batch [2256]	Speed: 246.480909 samples/sec	accuracy=0.177138
Epoch [0] Batch [2257]	Speed: 250.046441 samples/sec	accuracy=0.177170
Epoch [0] Batch [2258]	Speed: 252.617936 samples/sec	accuracy=0.177247
Epoch [0] Batch [2259]	Speed: 247.050744 samples/sec	accuracy=0.177257
Epoch [0] Batch [2260]	Speed: 249.712976 samples/sec	accuracy=0.177311
Epoch [0] Batch [2261]	Speed: 246.642518 samples/sec	accuracy=0.177277
Epoch 

Epoch [0] Batch [2365]	Speed: 247.942328 samples/sec	accuracy=0.179755
Epoch [0] Batch [2366]	Speed: 247.794382 samples/sec	accuracy=0.179763
Epoch [0] Batch [2367]	Speed: 248.422536 samples/sec	accuracy=0.179730
Epoch [0] Batch [2368]	Speed: 249.121930 samples/sec	accuracy=0.179759
Epoch [0] Batch [2369]	Speed: 253.764997 samples/sec	accuracy=0.179747
Epoch [0] Batch [2370]	Speed: 253.880968 samples/sec	accuracy=0.179798
Epoch [0] Batch [2371]	Speed: 249.881086 samples/sec	accuracy=0.179827
Epoch [0] Batch [2372]	Speed: 253.235645 samples/sec	accuracy=0.179857
Epoch [0] Batch [2373]	Speed: 248.203496 samples/sec	accuracy=0.179928
Epoch [0] Batch [2374]	Speed: 252.758752 samples/sec	accuracy=0.180000
Epoch [0] Batch [2375]	Speed: 252.166103 samples/sec	accuracy=0.180008
Epoch [0] Batch [2376]	Speed: 252.502355 samples/sec	accuracy=0.180017
Epoch [0] Batch [2377]	Speed: 249.544350 samples/sec	accuracy=0.180067
Epoch [0] Batch [2378]	Speed: 246.226966 samples/sec	accuracy=0.180160
Epoch 

Epoch [0] Batch [2482]	Speed: 248.583756 samples/sec	accuracy=0.183447
Epoch [0] Batch [2483]	Speed: 253.376385 samples/sec	accuracy=0.183394
Epoch [0] Batch [2484]	Speed: 252.815884 samples/sec	accuracy=0.183400
Epoch [0] Batch [2485]	Speed: 251.468381 samples/sec	accuracy=0.183467
Epoch [0] Batch [2486]	Speed: 251.698512 samples/sec	accuracy=0.183494
Epoch [0] Batch [2487]	Speed: 253.639929 samples/sec	accuracy=0.183581
Epoch [0] Batch [2488]	Speed: 247.173038 samples/sec	accuracy=0.183688
Epoch [0] Batch [2489]	Speed: 251.256732 samples/sec	accuracy=0.183675
Epoch [0] Batch [2490]	Speed: 249.329996 samples/sec	accuracy=0.183701
Epoch [0] Batch [2491]	Speed: 247.794382 samples/sec	accuracy=0.183708
Epoch [0] Batch [2492]	Speed: 246.786187 samples/sec	accuracy=0.183714
Epoch [0] Batch [2493]	Speed: 252.611850 samples/sec	accuracy=0.183681
Epoch [0] Batch [2494]	Speed: 245.823616 samples/sec	accuracy=0.183707
Epoch [0] Batch [2495]	Speed: 247.806826 samples/sec	accuracy=0.183714
Epoch 

MXNetError: [20:48:11] src/io/local_filesys.cc:166: Check failed: allow_null  LocalFileSystem: fail to open "model/model-0000.params"

Stack trace returned 10 entries:
[bt] (0) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x264b929) [0x7f99fa3cc929]
[bt] (1) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x26420aa) [0x7f99fa3c30aa]
[bt] (2) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/mxnet/libmxnet.so(MXNDArraySave+0x5a3) [0x7f99f9df0ac3]
[bt] (3) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/lib-dynload/_ctypes.so(ffi_call_unix64+0x4c) [0x7f9a3ebd935c]
[bt] (4) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/lib-dynload/_ctypes.so(ffi_call+0x1f5) [0x7f9a3ebd8ab5]
[bt] (5) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/lib-dynload/_ctypes.so(_ctypes_callproc+0x3e6) [0x7f9a3ebd0166]
[bt] (6) /home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/lib-dynload/_ctyp

In [5]:
# save params
net.save_params('{}/model-{:0>4}.params'.format(model_dir, epoch))


### CIFAR10 sample images

We'll use these CIFAR10 sample images to test the service:

<img style="display: inline; height: 32px; margin: 0.25em" src="images/airplane1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/automobile1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/bird1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/cat1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/deer1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/dog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/frog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/horse1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/ship1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/truck1.png" />

In [27]:
net = models.get_model('resnet34_v2', ctx=mx.cpu(), pretrained=False, classes=10)
net.load_params('{}/model-{:0>4}.params'.format(model_dir, 0), ctx=mx.gpu(0))

In [30]:
# load the CIFAR10 samples, and convert them into format we can use with the prediction endpoint
from cifar10_utils import read_images

filenames = ['images/airplane1.png',
             'images/automobile1.png',
             'images/bird1.png',
             'images/cat1.png',
             'images/deer1.png',
             'images/dog1.png',
             'images/frog1.png',
             'images/horse1.png',
             'images/ship1.png',
             'images/truck1.png']

image_data = read_images(filenames)


3072


In [33]:
data = gluon.utils.split_and_load(image_data[0], ctx_list=[mx.gpu(0)], batch_axis=0)
for x in data:
    y = net(x)
    print(y)


[[ 1.38848114  1.2084254  -0.32151598 -1.30662465 -0.43435436 -1.46848369
  -1.27514076 -0.7148152   1.50975513  1.62946868]]
<NDArray 1x10 @gpu(0)>


In [34]:
test_data = get_test_data(num_cpus, data_dir, 1, (3, 32, 32))
for i, batch in enumerate(test_data):    
    data = gluon.utils.split_and_load(batch.data[0], ctx_list=[mx.gpu(0)], batch_axis=0)
    for x in data:
        y = net(x)
        prediction = mx.nd.argmax(y, axis=1)
        print(prediction)
        break
    break
    


[ 5.]
<NDArray 1 @gpu(0)>
